## TensorFlow单隐层网络实现手写数字识别

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

### 5.1 MNIST数据处理

MNIST数据集是NIST数据集的一个子集，包含60000张图片作为训练数据，10000张图片作为测试数据，每一张图片代表0-9中的一个数字，图片大小28×28。  
在Yann LeCun教授的网站（http://yann.lecun.com/exdb/mnist ）中对MNIST数据集做了详细的介绍。

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

# 载入MNIST数据集
mnist=input_data.read_data_sets("/path/to/MNIST_data/",one_hot=True)
print("Training data size: ", mnist.train.num_examples)
print("Validating data size: ", mnist.validation.num_examples)
print("Testing data size: ", mnist.test.num_examples)

Extracting /path/to/MNIST_data/train-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /path/to/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/t10k-labels-idx1-ubyte.gz
Training data size:  55000
Validating data size:  5000
Testing data size:  10000


 input_data.read_data_sets函数生成的类会自动将数据集划分为3个子集：train、validation和test。

In [3]:
# 查看某张图片像素矩阵生成的的一维数组及其相应的标签
print("Example training data: ", mnist.train.images[0])
print("Example training data label: ", mnist.train.labels[0])

Example training data:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.       

mnist.train.next_batch函数可以从所有训练数据中读取一小部分作为一个训练batch。

In [4]:
# 使用mnist.train.next_batch函数
batch_size = 100
xs, ys = mnist.train.next_batch(batch_size)
# 从train的集合中选取batch_size个训练数据。
print("X shape:", xs.shape)
print("Y shape:", ys.shape)

X shape: (100, 784)
Y shape: (100, 10)


### 5.2 神经网络模型训练及不同模型结果对比

下面给出完整的TensorFlow程序解决MNIST手写体数字识别问题，用到了带指数衰减的学习率设置、正则化避免过拟合、滑动平均模型使模型更健壮。

1. 设置输入和输出节点的个数，配置神经网络的参数

In [9]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#MNIST数据集相关的常数
INPUT_NODE = 784     # 输入节点，等于图片的像素
OUTPUT_NODE = 10     # 输出节点，等于类别的数目

#配置神经网络的参数
LAYER1_NODE = 500    # 隐藏层结点数，这里使用只有一个隐藏层的网络结构作为样例
BATCH_SIZE = 100     # 一个训练batch中的的样本个数。
                     # 数字越小越接近随机梯度下降，越大越接近梯度下降
LEARNING_RATE_BASE = 0.8    # 基础的学习率
LEARNING_RATE_DECAY = 0.99    # 学习率的衰减率
REGULARAZTION_RATE = 0.0001   # 描述模型复杂度的正则化项在损失函数中的系数
TRAINING_STEPS = 30000        # 训练轮数
MOVING_AVERAGE_DECAY = 0.99  # 滑动平均衰减率    

2. 定义辅助函数来计算前向传播结果，定义使用ReLU做为激活函数的三层全连接神经网络

In [10]:
def inference(input_tensor, avg_class, 
              weights1, biases1, 
              weights2, biases2):
    # 当没有提供滑动平均类时，直接使用参数当前的取值
    if avg_class == None:
        # 计算隐藏层的前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        # 因为在计算损失函数时会一并计算softmax函数，所以这里不需要加入
        # 因为预测是使用的是输出值相对大小，所以softmax层也可以不加入
        return tf.matmul(layer1, weights2) + biases2
    
    else:
        # 使用avg_class.average函数来计算得出变量的滑动平均值
        # 然后计算相应的神经网络前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + 
                            avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

3. 训练模型的过程

In [11]:
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    
    # 生成隐藏层的参数。
    weights1 = tf.Variable(
        tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层的参数。
    weights2 = tf.Variable(
        tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    # 计算不含滑动平均类的前向传播结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义存储训练轮数的变量
    # 这个变量不需要计算滑动平均值，所以指定为不可训练的比俺俩
    # 在使用TF训练神经网络时，一般会将代表训练轮数的变量指定为不可训练的参数
    global_step = tf.Variable(0, trainable=False)
    
    # 给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类
    variable_averages = tf.train.ExponentialMovingAverage(
        MOVING_AVERAGE_DECAY, global_step)
    
    # 在所有代表神经网络参数的变量上使用滑动平均
    # tf.trainable_variables返回的是图上集合GraphKeys.TRAINABLE_VARIABLES中的元素，这个集合的元素
    # 就是所有没有指定trainable=Flase的参数
    variables_averages_op = variable_averages.apply(
        tf.trainable_variables())
    
    # 计算使用了滑动平均之后的前向传播结果
    average_y = inference(
        x, variable_averages, weights1, biases1, weights2, biases2)
    
    # 计算交叉熵及其平均值
    # tf.argmax函数得到正确答案对应的类别编号
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=y, labels=tf.argmax(y_, 1))
    #计算当前batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # L2正则化损失函数的计算
    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    # 计算模型的正则化损失，一般只计算神经网络边上权重的正则化损失，而不使用偏置项
    regularization = regularizer(weights1) + regularizer(weights2)
    # 总损失等于交叉熵损失和正则化损失之和
    loss = cross_entropy_mean + regularization
    # 设置指数衰减的学习率。
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE, # 基础的学习率
        global_step, # 当前迭代的轮数
        mnist.train.num_examples / BATCH_SIZE, # 过完所有训练数据需要的迭代次数 
        LEARNING_RATE_DECAY, # 学习率衰减速度
        staircase=True)
    
    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # 反向传播更新参数和更新每一个参数的滑动平均值
    # tf支持进行一次完成多个操作,提供了tf.control_dependencies和tf.group两种机制
    # 例如创建一个group，把train_step和variables_averages_op两个操作放在一起进行，等同于以下操作：
    # with tf.control_dependencies([train_step, variables_averages_op]):
    #     train_op = tf.no_op(name='train')
    train_op = tf.group(train_step, variables_averages_op)    
    
    # 计算正确率
    # average_y.shape = [None, OUTPUT_NODE]，tf.argmax(average_y, 1)表示返回average_y中最大值的序号
    # Signature: tf.argmax(input, axis=None, name=None, dimension=None, output_type=tf.int64)
    # Returns the index with the largest value across axes of a tensor. (deprecated arguments)
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    # 将布尔型数值转换为实数型，然后计算平均值。这个平均值就是模型在这一组数据上的正确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化会话并开始训练过程。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 准备验证数据
        validate_feed = {x: mnist.validation.images, 
                         y_: mnist.validation.labels}
        # 准备测试数据。在真实应用中，这部分数据在测试时是不可见的，这个数据只是作为模型优劣的最后评价标准
        test_feed = {x: mnist.test.images, y_: mnist.test.labels} 
        
        # 迭代地训练神经网络。
        for i in range(TRAINING_STEPS):
            # 每1000轮输出一次在验证数据集上的测试结果
            if i % 1000 == 0:
                # 计算滑动平均模型在验证数据上的结果
                # 因为MNIST数据集比较小，所以一次可以处理所有的验证数据，而不需要划分更小的batch
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy "
                      "using average model is %g " % (i, validate_acc))
                
            # 产生这一轮使用的一个batch的训练数据，并运行训练过程
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x:xs,y_:ys})
            
        # 在训练结束之后，在测试数据上检测神经网络模型的最终正确率
        test_acc=sess.run(accuracy,feed_dict=test_feed)
        print("After %d training step(s), test accuracy using average "
              "model is %g" %(TRAINING_STEPS, test_acc))

4. 主程序入口

In [12]:
def main(argv=None):
    # 声明处理MNIST数据集的类，这个类在初始化时会自动下载数据
    mnist = input_data.read_data_sets("/tmp/data", one_hot=True)
    train(mnist)

# TensorFlow提供的一个主程序入口，tf.app.run会调用上面定义的main函数
if __name__=='__main__':
    tf.app.run()

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
After 0 training step(s), validation accuracy using average model is 0.1246 
After 1000 training step(s), validation accuracy using average model is 0.976 
After 2000 training step(s), validation accuracy using average model is 0.9822 
After 3000 training step(s), validation accuracy using average model is 0.9824 
After 4000 training step(s), validation accuracy using average model is 0.9832 
After 5000 training step(s), validation accuracy using average model is 0.9832 
After 6000 training step(s), validation accuracy using average model is 0.9842 
After 7000 training step(s), validation accuracy using average model is 0.984 
After 8000 training step(s), validation accuracy using average model is 0.9832 
After 9000 training step(s), validation accuracy using average model is 0.9844 
After 10000 tra

SystemExit: 

C:\Users\74575\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


从结果看出，从4000轮开始，模型在验证数据集上的表现开始波动，说明模型已经接近极小值，迭代也可以结束了。

除了使用验证数据集，还可以采用交叉验证（cross validation），但因为神经网络训练时间本身就比较长，采用cross validation会花费大量时间，所以在海量数据的情况下，一般会更多地采用验证数据集的形式来评测模型的效果。

在这里，一个模型在MNIST测试数据集上的正确率将简称为“正确率”。  
前面提到了设计神经网络时的5种优化方法：  
在神经网络结构的设计上，需要使用激活函数和多层隐藏层；  
在神经网络优化时，可以使用指数衰减的学习率、加入正则化的损失函数、滑动平均模型。  
使用所有优化、不用滑动平均、不用正则化、不用指数衰减学习率、不用隐藏层、不用激活函数（学习率改为0.05）的正确率分别为：  
0.9841、0.9839、0.9831、0.9838、0.9256、0.9257（10次运行的平均值）  
可以发现神经网络的结构对最终模型的效果有本质性的影响。  
当问题更加复杂时，滑动平均模型和指数衰减的学习率可以发挥更大作用：例如在CIFAR-10图像分类数据集上，使用滑动平均模型可以将错误率降低11%，使用指数衰减的学习率可以将错误率降低7%。  
加入正则化的损失函数给模型效果带来的提升要相对显著。

由于MNIST问题本身相对简单，优化方法提升效果不明显，但当需要解决的问题和模型更加复杂时，这些优化方法将产生更大影响。